In [ ]:
import cv2
import face_recognition
import os
import numpy as np

# Cargar las imágenes conocidas y sus nombres
def load_known_faces(known_faces_dir):
    """
    Carga imágenes conocidas desde un directorio, las codifica y extrae los nombres.
    
    Args:
    - known_faces_dir (str): Ruta al directorio que contiene las imágenes conocidas.

    Returns:
    - known_face_encodings (list): Lista de codificaciones faciales.
    - known_face_names (list): Lista de nombres de las personas correspondientes a las codificaciones.
    """
    known_face_encodings = []
    known_face_names = []

    # Recorre todas las imágenes en el directorio especificado
    for filename in os.listdir(known_faces_dir):
        if filename.lower().endswith((".jpg", ".png")):
            image_path = os.path.join(known_faces_dir, filename)
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            # Si se encuentran caras, guarda la codificación y el nombre (nombre del archivo sin extensión)
            if face_encodings:
                print(f"Archivo leído: {filename}")
                known_face_encodings.append(face_encodings[0])
                known_face_names.append(os.path.splitext(filename)[0])
            else:
                print(f"No se encontraron caras en la imagen: {filename}")

    return known_face_encodings, known_face_names

# Configurar el reconocimiento facial
def recognize_faces_from_webcam(known_face_encodings, known_face_names):
    """
    Inicia el reconocimiento facial utilizando la cámara web, comparando las caras detectadas
    con las codificaciones faciales conocidas.
    
    Args:
    - known_face_encodings (list): Lista de codificaciones faciales conocidas.
    - known_face_names (list): Lista de nombres asociados a las codificaciones.
    """
    video_capture = cv2.VideoCapture(0)

    if not video_capture.isOpened():
        print("Error: No se pudo abrir la cámara.")
        return

    print("Iniciando reconocimiento facial. Presiona 'q' para salir.")

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Error: No se pudo leer un fotograma de la cámara.")
            break
    
        # Convertir el fotograma a formato RGB (necesario para face_recognition)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detectar las ubicaciones de las caras en el fotograma
        face_locations = face_recognition.face_locations(rgb_frame)
        if face_locations:
            # Codificar las caras detectadas en el fotograma
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            # Comparar cada cara detectada con las caras conocidas
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # Comparar la cara detectada con las codificaciones conocidas
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Desconocido"
                
                # Usar la menor distancia como criterio de coincidencia (más preciso que solo comparar booleanos)
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                
                if matches[best_match_index]:
                    # Si se encuentra una coincidencia, asignar el nombre correspondiente
                    name = known_face_names[best_match_index]
                    # Calcular el porcentaje de similitud basado en la distancia (1 - distancia)
                    similarity_percentage = (1 - face_distances[best_match_index]) * 100
                    # Agregar el porcentaje al nombre
                    name += f" ({similarity_percentage:.2f}%)"
    
                # Dibujar un recuadro alrededor de la cara detectada
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                # Dibujar el nombre y el porcentaje de similitud debajo de la cara
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
        else:
            print("No se detectaron caras en el frame.")

        # Mostrar el resultado en la ventana de video
        cv2.imshow('Video', frame)

        # Presionar 'q' para salir del bucle
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar la ventana
    video_capture.release()
    cv2.destroyAllWindows()

# Cargar las caras conocidas desde una carpeta
known_faces_dir = "caras"  # Especificar la ruta del directorio donde están las imágenes conocidas
known_face_encodings, known_face_names = load_known_faces(known_faces_dir)

# Ejecutar el reconocimiento facial desde la webcam
recognize_faces_from_webcam(known_face_encodings, known_face_names)
